In [1]:
import pandas as pd

df = pd.read_parquet("data/oxford-iiit-pet/evaluation_samples.parquet")
df


,filepath,label
0,data/oxford-iiit-pet/22/newfoundland_31.jpg,newfoundland
1,data/oxford-iiit-pet/25/pug_57.jpg,pug
2,data/oxford-iiit-pet/1/american_bulldog_80.jpg,american_bulldog
3,data/oxford-iiit-pet/15/great_pyrenees_22.jpg,great_pyrenees
4,data/oxford-iiit-pet/16/havanese_97.jpg,havanese
...,...,...
3664,data/oxford-iiit-pet/10/chihuahua_91.jpg,chihuahua
3665,data/oxford-iiit-pet/17/japanese_chin_83.jpg,japanese_chin
3666,data/oxford-iiit-pet/28/saint_bernard_88.jpg,saint_bernard
3667,data/oxford-iiit-pet/21/miniature_pinscher_32.jpg,miniature_pinscher


In [2]:
from active_vision import ActiveLearner
from fastai.vision.models.all import resnet18

al = ActiveLearner("convnext_small_in22k")

2025-01-24 17:09:08.337 | INFO     | active_vision.core:load_model:45 - Loading timm model convnext_small_in22k


In [3]:
from fastai.vision.all import aug_transforms

al.load_dataset(df, 
                filepath_col="filepath", 
                label_col="label", 
                batch_size=64,
                valid_pct=0.35,
                image_size=400, 
                batch_tfms=aug_transforms(size=224, min_scale=0.75)
                )

# al.load_dataset(df, 
#                 filepath_col="filepath", 
#                 label_col="label", 
#                 batch_size=64,
#                 image_size=400, 
#                 batch_tfms=aug_transforms(size=224, min_scale=0.75),
#                 learner_path="convnext_small_in22k_pets.pkl")

2025-01-24 17:09:08.344 | INFO     | active_vision.core:load_dataset:59 - Loading dataset from filepath and label
2025-01-24 17:09:08.345 | INFO     | active_vision.core:load_dataset:61 - Creating dataloaders
2025-01-24 17:09:08.769 | INFO     | active_vision.core:load_dataset:83 - Creating learner
/home/dnth/Desktop/active-vision/.venv/lib/python3.12/site-packages/timm/models/_factory.py:126: UserWarning: Mapping deprecated model name convnext_small_in22k to current convnext_small.fb_in22k.
  model = create_fn(
2025-01-24 17:09:09.517 | INFO     | active_vision.core:load_dataset:92 - Done. Ready to train.


In [4]:
al.train_set

,filepath,label
2685,data/oxford-iiit-pet/2/american_pit_bull_terrier_40.jpg,american_pit_bull_terrier
318,data/oxford-iiit-pet/4/beagle_38.jpg,beagle
908,data/oxford-iiit-pet/4/beagle_71.jpg,beagle
3622,data/oxford-iiit-pet/9/British_Shorthair_86.jpg,british_shorthair
2205,data/oxford-iiit-pet/35/wheaten_terrier_200.jpg,wheaten_terrier
...,...,...
2031,data/oxford-iiit-pet/14/german_shorthaired_85.jpg,german_shorthaired
2119,data/oxford-iiit-pet/36/yorkshire_terrier_198.jpg,yorkshire_terrier
3596,data/oxford-iiit-pet/14/german_shorthaired_9.jpg,german_shorthaired
255,data/oxford-iiit-pet/16/havanese_90.jpg,havanese


In [5]:
# al.valid_set

In [6]:
# al.show_batch(num_samples=9, num_cols=3, unique=False)

In [7]:
# al.lr_find()

In [8]:
# al.train(epochs=5, lr=8e-3, head_tuning_epochs=1)

In [9]:
# evaluation_df = pd.read_parquet("data/oxford-iiit-pet/evaluation_samples.parquet")
# al.evaluate(evaluation_df, filepath_col="filepath", label_col="label")

In [10]:
df = pd.read_parquet("data/oxford-iiit-pet/unlabeled_samples.parquet")
filepaths = df["filepath"].tolist()
len(filepaths)

3310

In [11]:
# # Outlier image paths

# # get all images in a dir

# import glob
# import os

# directory = '/home/dnth/Downloads/sample_images/coco/'  
# image_extensions = ['*.jpg', '*.jpeg', '*.png', '*.gif', '*.bmp', '*.webp']

# # Collect all image files matching the extensions
# image_files = []
# for ext in image_extensions:
#     image_files.extend(glob.glob(os.path.join(directory, ext)))

# print(image_files)  # List of image file paths

In [12]:
pred_df = al.predict(filepaths, batch_size=128)

2025-01-24 17:09:09.566 | INFO     | active_vision.core:predict:139 - Running inference on 3310 samples


In [13]:
uncertain_df = al.sample_uncertain(pred_df, num_samples=3000, strategy="ratio-of-confidence")


2025-01-24 17:09:16.123 | INFO     | active_vision.core:sample_uncertain:224 - Using ratio of confidence strategy to get top 3000 samples


In [14]:
uncertain_df

,filepath,pred_label,pred_conf,score,probs,logits
1086,data/oxford-iiit-pet/11/Egyptian_Mau_18.jpg,american_bulldog,0.4645,1.0000,"[9.319339255853265e-08, 0.4645370543003082, 0.05091226473450661, 1.3040101976002916e-07, 1.4776371415337053e-07, 3.350475162733346e-05, 2.3337477728091471e-07, 9.552514711685944e-06, 5.961213173577562e-05, 2.0343159448654724e-11, 2.2660296750132147e-08, 0.0034917963203042746, 6.174514419399202e-05, 0.0003344436699990183, 1.8696053413107608e-14, 1.5881353121471875e-08, 5.135819325374769e-09, 4.4041160435881466e-05, 0.00014986487803980708, 0.010025219060480595, 0.4645370543003082, 4.7605635700165294e-07, 1.0644608892107499e-06, 8.555738168070093e-05, 0.0002456418878864497, 2.466869045747444e...","[-5.078125, 10.34375, 8.1328125, -4.7421875, -4.6171875, 0.806640625, -4.16015625, -0.4482421875, 1.3828125, -13.5078125, -6.4921875, 5.453125, 1.41796875, 3.107421875, -20.5, -6.84765625, -7.9765625, 1.080078125, 2.3046875, 6.5078125, 10.34375, -3.447265625, -2.642578125, 1.744140625, 2.798828125, 0.50048828125, -6.65234375, -3.16796875, 2.28515625, -11.625, -4.234375, -17.734375, -5.765625, 4.6640625, -14.015625, 2.400390625, 5.46875]"
2141,data/oxford-iiit-pet/27/Russian_Blue_177.jpg,egyptian_mau,0.3603,1.0000,"[6.243108252590446e-09, 0.1245318353176117, 0.0715126171708107, 1.2637966051443073e-07, 7.78370090603886e-10, 0.0016687273746356368, 3.1524595556220447e-07, 4.778130247018453e-08, 0.00019054622680414468, 1.673758798226288e-09, 6.429677057440131e-08, 0.360344797372818, 2.175456302211387e-06, 7.696464763284894e-07, 1.5112124876134891e-13, 7.270477109244666e-09, 2.2354285533765506e-07, 7.631393964402378e-05, 2.1170932086533867e-05, 2.9050490411464125e-05, 0.066138356924057, 5.199079168960452e-05, 2.3154157702265366e-07, 3.0653482099296525e-05, 0.00017180826398544014, 0.00033770082518458366, 8...","[-7.54296875, 9.265625, 8.7109375, -4.53515625, -9.625, 4.953125, -3.62109375, -5.5078125, 2.783203125, -8.859375, -5.2109375, 10.328125, -1.689453125, -2.728515625, -18.171875, -7.390625, -3.96484375, 1.8681640625, 0.5859375, 0.90234375, 8.6328125, 1.484375, -3.9296875, 0.9560546875, 2.6796875, 3.35546875, -4.9765625, -6.6875, 7.0703125, -8.8359375, -3.431640625, -11.953125, -5.5625, 4.0546875, -12.6953125, -1.6748046875, 10.328125]"
1569,data/oxford-iiit-pet/25/pug_143.jpg,american_pit_bull_terrier,0.4140,1.0000,"[3.1083795259956304e-11, 0.0010839462047442794, 0.4140227138996124, 1.9847242072046356e-07, 4.513167183722855e-12, 6.585353276022943e-06, 1.0191984756158234e-10, 1.388062376150856e-08, 0.00017625950567889959, 2.9535911477002186e-13, 2.48343207109869e-10, 0.009511326439678669, 4.5348915591603145e-06, 1.5963289001774683e-07, 1.4990619693917157e-16, 5.175916917288248e-10, 5.675234859836564e-08, 1.1947847269766498e-05, 0.0009908070787787437, 7.640283729415387e-05, 0.4140227138996124, 3.208486631933738e-08, 8.142860274951147e-10, 5.230613169260323e-05, 0.02077464573085308, 9.180011693388224e-05...","[-10.1171875, 7.25, 13.1953125, -1.35546875, -12.046875, 2.146484375, -8.9296875, -4.015625, 5.43359375, -14.7734375, -8.0390625, 9.421875, 1.7734375, -1.5732421875, -22.359375, -7.3046875, -2.607421875, 2.7421875, 7.16015625, 4.59765625, 13.1953125, -3.177734375, -6.8515625, 4.21875, 10.203125, 4.78125, -7.06640625, -6.08984375, 7.22265625, -14.1875, -2.05078125, -10.53125, -4.80078125, 10.703125, -6.83984375, -0.71484375, 11.8125]"
36,data/oxford-iiit-pet/25/pug_155.jpg,maine_coon,0.3946,1.0000,"[8.14473868123583e-13, 0.005944454111158848, 0.07413563132286072, 5.514297907183163e-09, 4.210218285116696e-11, 1.0163691399611707e-07, 2.1672346209311577e-11, 6.078159503486447e-11, 0.0001767239737091586, 3.7211017958355824e-14, 1.6393575386075554e-10, 0.0045935180969536304, 3.0946262086217757e-06, 3.313571141916327e-06, 7.239756036305016e-17, 4.311347723273684e-09, 9.1418677072852e-08, 8.218427683459595e-05, 2.7524174583959393e-06, 1.6057107131928205e-05, 0.3945600092411041, 3.069627041440981e-08, 1.755866207986

In [15]:
# diverse_df = al.sample_diverse(pred_df, num_samples=100, strategy="model-based-outlier")
# diverse_df

In [16]:
al.label(uncertain_df, output_filename="active_labeled_testing")

2025-01-24 17:09:17.196 | INFO     | active_vision.core:label:362 - Launching labeling interface for 3000 samples


* Running on local URL:  http://127.0.0.1:7860

To create a public link, set `share=True` in `launch()`.
